In [ ]:
# reloading magic
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches
import seaborn as sns
sns.set(style="white", palette="muted")
sns.set_context("talk", font_scale=1.4)
%matplotlib qt

In [ ]:
import matplotlib as mpl
mpl.rcParams['font.family'] = 'Helvetica'
mpl.rcParams['font.weight'] = 'light'
mpl.rcParams['axes.labelweight'] = 'light'


In [ ]:
# linux
comm_path = os.path.normpath('/home/oliver/repos/bikesim/comm')
scripts_path = os.path.normpath('/home/oliver/repos/bikesim/scripts')

In [ ]:
# osx
comm_path = os.path.normpath('/Users/oliverlee/repos/bikesim/comm')
scripts_path = os.path.normpath('/Users/oliverlee/repos/bikesim/scripts')

In [ ]:
for d in (comm_path, scripts_path):
    if d not in sys.path:
        sys.path.append(d)
from parse_data import *

In [ ]:
log_dirname = '/Users/oliverlee/repos/bikesim/comm/data/exp0'
if log_dirname is None:
    log_dirname = comm_path

In [ ]:
subject_map = parse_log_dir(log_dirname)

In [ ]:
df = balance_df(subject_map.values())
full_code = df.subject + '.' + df.torque_enabled.astype(int).astype(str)
df['full_subject_code'] = full_code
plt.subplots()
g = sns.boxplot(df.log_timespan, df.full_subject_code, color="Paired")
g.set_xlabel('subject')

In [ ]:
g1 = plot_dist_grouped_boxchart(subject_map)
plt.tight_layout()
plt.savefig('grouped_boxcharts.pdf', format='pdf', dpi=1000)
size = plt.gcf().get_size_inches()
# g1.fig.suptitle('Distribution of time balanced per subject', fontsize=18)

In [ ]:
fig2, ax2 = plot_dist_overlapping_histogram(subject_map)
plt.gcf().set_size_inches(size[0], size[1], forward=True)
plt.tight_layout()
plt.savefig('overlapping_histogram.pdf', format='pdf', dpi=1000)
# plt.subplots_adjust(top=0.95)
# fig2.suptitle('Distribution of time balanced for all subjects', fontsize=18)

In [ ]:
plt.close('all')
g3 = plot_subject_balance_time_change_boxplot(subject_map)
# plt.subplots_adjust(top=0.9)
plt.gcf().set_size_inches(size[0], size[1], forward=True)
plt.tight_layout()
plt.savefig('subject_change_boxplot.pdf', format='pdf', dpi=1000)
# g3.fig.suptitle('Change in time balanced per subject', fontsize=18)

In [ ]:
plt.close('all')
fig4, ax4 = plot_overlapping_psd(subject_map, 'phi', 'longest')
plt.gcf().set_size_inches(size[0], size[1], forward=True)
plt.tight_layout()
# plt.savefig('psd_lean_longest.pdf', format='pdf', dpi=1000)
# fig4.suptitle('Power spectral density of lean angle of longest run', fontsize=18)

In [ ]:
plt.close('all')
fig5, ax5 = plot_overlapping_psd(subject_map, 'phi', 'all')
plt.gcf().set_size_inches(size[0], size[1], forward=True)
plt.tight_layout()
plt.savefig('psd_lean_all.pdf', format='pdf', dpi=1000)
# fig5.suptitle('Power spectral density of lean angle of all runs', fontsize=18)

In [ ]:
fig6, ax6 = plot_overlapping_psd(subject_map, 'deltad', 'longest')
plt.gcf().set_size_inches(size[0], size[1], forward=True)
plt.tight_layout()
# plt.savefig('psd_steer_rate_longest.pdf', format='pdf', dpi=1000)
# fig6.suptitle('Power spectral density of steer rate of longest run', fontsize=18)

In [ ]:
plt.close('all')
fig7, ax7 = plot_overlapping_psd(subject_map, 'deltad', 'all')
# ax7.set_rasterized(True)
plt.gcf().set_size_inches(size[0], size[1], forward=True)
plt.tight_layout()
plt.savefig('psd_steer_rate_all.pdf', format='pdf', dpi=1000)
# fig7.suptitle('Power spectral density of steer rate of all run', fontsize=18)

In [ ]:
plt.close('all')
fig8, ax8 = plot_overlapping_psd(subject_map, 'delta', 'all')
# ax7.set_rasterized(True)
plt.gcf().set_size_inches(size[0], size[1], forward=True)
plt.tight_layout()
fig8.suptitle('Power spectral density of steer of all runs', fontsize=18)
plt.savefig('psd_steer_all.pdf', format='pdf', dpi=1000)

In [ ]:
l = subject_map['003'].logs[-1]
print('feedback: {}'.format(l.feedback))
f, axes = plot_lean_steer_yy(l.sensor, l.actuator)
f, axes = plot_subplots(l.sensor, l.actuator, ('phi', 'delta', 'deltad'))

fig, ax = plt.subplots()
f, psds = signal.welch(l.actuator.get_field('phi'), 50, nperseg=256, return_onesided=True)
ax.loglog(f, psds, label='lean')
f, psds = signal.welch(l.sensor.get_field('delta'), 50, nperseg=256, return_onesided=True)
ax.loglog(f, psds, label='steer')
f, psds = signal.welch(l.sensor.get_field('deltad'), 50, nperseg=256, return_onesided=True)
ax.loglog(f, psds, label='steer rate')
ax.set_xlim([f[0], f[-1]])
ax.set_xlabel('frequency [Hz]')
ax.legend()
fig.suptitle('psd')

In [ ]:
for s in subject_map.values():
    print(len(s.logs))

In [ ]:
sns.palplot(sns.color_palette())
colors = sns.color_palette()
print(colors[2])